In [3]:
from langchain.llms import GooglePalm
from hello import api_key
import os
os.environ['GOOGLE_AI_KEY']=api_key
llm=GooglePalm(google_api_key=api_key,temperature=0.6)

In [4]:
import pandas as pd
from langchain.document_loaders.csv_loader import CSVLoader


loader = CSVLoader(file_path="project.csv", source_column="States + UTs")

data = loader.load()





In [7]:
import streamlit as st
import os
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import google.generativeai as genai
from langchain.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import RetrievalQA

from langchain.prompts import PromptTemplate

embeddings = GoogleGenerativeAIEmbeddings(google_api_key=api_key,model="models/embedding-001")
e = embeddings.embed_query("states")
len(e)

instructor_embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001", task_type="retrieval_query"
)
vectordb = FAISS.from_documents(documents=data,embedding=instructor_embeddings)


retriever = vectordb.as_retriever(score_threshold = 0.7)
rdocs = retriever.get_relevant_documents("Goa")
rdocs


from langchain.prompts import PromptTemplate

prompt_template = """Given the following context and a question, generate an answer based on this context only.
In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.

CONTEXT: {context}

QUESTION: {question}"""


PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}
chain = RetrievalQA.from_chain_type(llm=llm,chain_type="stuff",retriever=retriever, input_key="query",return_source_documents=True, chain_type_kwargs=chain_type_kwargs)
chain("spirtuality of Goa and its origin")




ValidationError: 1 validation error for GoogleGenerativeAIEmbeddings
__root__
  Did not find google_api_key, please add an environment variable `GOOGLE_API_KEY` which contains it, or pass `google_api_key` as a named parameter. (type=value_error)